In [1]:
import warnings
warnings.filterwarnings("ignore")
import torch
import sys

import os
import numpy as np
from tqdm import tqdm
import torchaudio
import librosa
import glob

sys.path.append("../../cuhksz-phd/sho_util/pyfiles/")
from pytorch import cuda2numpy, cuda2cpu
from basic import plot_spectrogram
from sound import play_audio

sys.path.append("../")
from pyfiles.processsound import trim_audio_and_save

sys.path.append("./../../BigVGAN/")
import bigvgan
from meldataset import get_mel_spectrogram

In [8]:
data_dir = "/mntcephfs/lab_data/shoinoue/"
dataset_dir = data_dir + "Dataset/CMU-ARCTIC/"
fs = 16000
trim_threshold_in_db = 30
speakers = ["SLT"]

# modelpath = "/mntcephfs/data/audiow/shoinoue/Model/hf_hub/bigvgan/models--nvidia--bigvgan_v2_24khz_100band_256x/snapshots/61df17db326f0876b7201d7a56c831898c836ef4"
modelpath = "/mntcephfs/data/audiow/shoinoue/Model/hf_hub/bigvgan/models--nvidia--bigvgan_v2_22khz_80band_fmax8k_256x/snapshots/189a02ed3b7957e8534b40e6314262df53536ece"
model = bigvgan.BigVGAN.from_pretrained(modelpath, use_cuda_kernel=False)
model.remove_weight_norm()
model = model.eval()

def get_mel(path):
    trim_audio_and_save(path, savepath=tempfile, trim_threshold_in_db=trim_threshold_in_db)
    wav, sr = librosa.load(tempfile, sr=model.h.sampling_rate, mono=True) 
    wav = torch.FloatTensor(wav).unsqueeze(0)
    mel = get_mel_spectrogram(wav, model.h)
    mel = np.array(mel[0])
    return mel

Loading config.json from local directory
Loading weights from local directory
Removing weight norm...


In [12]:
save = True
tempfile = "temp2.wav"

for spk in speakers:
    print(spk)
    mel_dir = f"{dataset_dir}{spk}/80mel/"
    os.makedirs(mel_dir, exist_ok=True)
    filenames = glob.glob(dataset_dir + spk + "/wav/*")
    filenames.sort()
    for path in tqdm(filenames):
        trim_audio_and_save(path, savepath=tempfile, trim_threshold_in_db=trim_threshold_in_db)
            
        # mel
        mel = get_mel(tempfile)
        savepath = mel_dir + os.path.basename(path)[:-4] + ".npy"
        if save:
            np.save(savepath, mel)

SLT


100%|██████████| 1132/1132 [01:40<00:00, 11.30it/s]
